# A Simple Calculator

In this notebook, we implement a **symbolic calculator** using `Lezer`. The primary objective is to demonstrate the direct translation of a formal grammar definition into a working parser.

The language supports basic arithmetic operations ($+, -, *, /$), parenthesized expressions, and variable assignments. Formally, the grammar is defined as:

$$
\begin{array}{lcl}
 \texttt{stmnt}   & \rightarrow & \;\texttt{IDENTIFIER}\; \texttt{':='} \; \;\texttt{expr}\; \texttt{';'} \\
                  & \mid        & \;\texttt{expr}\; \texttt{';'} \\[0.2cm]   
 \texttt{expr}    & \rightarrow & \;\texttt{expr}\; \texttt{'+'} \; \texttt{product}  \\
                  & \mid        & \;\texttt{expr}\; \texttt{'-'} \; \texttt{product}  \\
                  & \mid        & \;\texttt{product}                                  \\[0.2cm]
 \texttt{product} & \rightarrow & \;\texttt{product}\; \texttt{'*'} \;\texttt{factor} \\
                  & \mid        & \;\texttt{product}\; \texttt{'/'} \;\texttt{factor} \\
                  & \mid        & \;\texttt{factor}                                   \\[0.2cm]
 \texttt{factor}  & \rightarrow &   \texttt{'('} \; \texttt{expr} \;\texttt{')'}      \\
                  & \mid        & \;\texttt{NUMBER}                                   \\
                  & \mid        & \;\texttt{IDENTIFIER}                               
\end{array}
$$

## Imports and Setup

We utilize `@lezer/generator` to compile our grammar and `@lezer/common` for tree navigation.

In [ ]:
import { buildParser } from "@lezer/generator";
import { TreeCursor }  from "@lezer/common";

## Grammar Definition

The Lezer grammar below maps **1:1** to the EBNF specification above.

To handle operator precedence correctly, we structure the grammar hierarchically:
1.  **`Expr`**: Handles lowest precedence operators (Additive: $+$, $-$).
2.  **`Product`**: Handles higher precedence operators (Multiplicative: $*$, $/$).
3.  **`Factor`**: Handles atomic units (Numbers, Identifiers) and highest precedence grouping (Parentheses).

We define `Program` as a sequence of one or more statements (`Statement+`). We chose this structure to simulate an interactive session where multiple statements are executed back-to-back. Since we cannot evaluate line-by-line interactively in this specific notebook environment, parsing a full block of statements allows us to demonstrate state persistence and sequential execution in a single pass.

In [ ]:
const grammarDefinition = `
    @top Program { Statement+ }

    @tokens {
        "+" "-" "*" "/" ":="
        Number      { $[0-9]+ ("." $[0-9]*)?   }
        Identifier  { $[a-zA-Z] $[a-zA-Z0-9_]* }
        space       { $[ \t\r\n]+ }
    }

    Statement {
        Identifier ":=" Expr ";" |
        Expr ";"
    }

    Expr {
        Expr "+" Product |
        Expr "-" Product |
        Product
    }

    Product {
        Product "*" Factor |
        Product "/" Factor |
        Factor
    }

    Factor {
        "(" Expr ")" |
        Number       |
        Identifier
    }

    @skip { space }
`;

In [ ]:
const parser = buildParser(grammarDefinition);

## Abstract Syntax Tree (AST) Configuration

We define the `AST` type as a recursive union type. Nodes can be primitive values (numbers, strings) or tuples representing operations. The tuple structure `[Operator, Operand, ...]` allows for a strictly typed yet flexible tree representation.

In [ ]:
type Operator = string;
type AST = string | number | 
           [Operator, AST]                |
           [Operator, AST, AST]           |
           [Operator, AST, AST, AST]      |
           [Operator, AST, AST, AST, AST];

To utilize the generic `cst2ast` transformer, we must configure which tokens act as **operators** (pivots) and which nodes represent **lists**. 

* **Operators**: These tokens promote their parent node to an operation node (e.g., seeing `+` turns the surrounding `Expr` node into a `['+', left, right]` tuple).
* **List Variables**: We designate `Program` as a list variable. This instructs the transformer to convert the iterative `Statement+` sequence into a recursive `['.', Head, Tail]` linked list structure.

In [ ]:
const operators = [":=", "+", "-", "*", "/"];
const listVars = ["Program"];

## The Evaluator

The evaluator consists of a state container (Environment) and a recursive evaluation function.

### Environment State
We use a `Map` to store variable assignments. This acts as the runtime memory for our calculator.

In [ ]:
const environment = new Map<string, number>();

### Node Evaluation Logic
The `evaluateNode` function processes a single AST node. It handles three distinct cases:

1.  **Literals:** Returns numeric values directly.
2.  **Variables:** Performs a lookup in the `environment`. Throws a `Runtime Error` if the variable is undefined.
3.  **Operations:** Recursively evaluates operands based on the operator.
    * **Assignment (`:=`):** Updates the environment.
    * **Arithmetic:** Performs calculation. Note the explicit check for **Division by Zero**, which prevents JavaScript's default `Infinity` behavior and ensures mathematical correctness.

We also implement strict type checking and structural validation to ensure that invalid AST shapes (e.g., missing operands) result in clear error messages rather than silent failures.

In [ ]:
function evaluateNode(node: AST): number {
    // 1. Literal (Number)
    if (typeof node === 'number') return node;

    // 2. Variable (Identifier)
    if (typeof node === 'string') {
        const val = environment.get(node);
        if (val === undefined) throw new Error(`Runtime Error: Undefined variable '${node}'`);
        return val;
    }

    // 3. Operation (Tuple)
    if (Array.isArray(node)) {
        // We extract elements but verify their existence before use to satisfy strict typing.
        const op = node[0];
        const left = node[1];
        const right = node[2];

        // Case A: Assignment ":="
        if (op === ":=") {
            if (typeof left === 'string' && right !== undefined) {
                const val = evaluateNode(right);
                environment.set(left, val);
                return val;
            }
            throw new Error("Runtime Error: Invalid assignment. Expected 'Identifier := Expr'.");
        }

        // Case B: Arithmetic Operations
        if (["+", "-", "*", "/"].includes(op as string)) {
            // Ensure both operands exist
            if (left !== undefined && right !== undefined) {
                const lVal = evaluateNode(left);
                const rVal = evaluateNode(right);

                switch (op) {
                    case "+": return lVal + rVal;
                    case "-": return lVal - rVal;
                    case "*": return lVal * rVal;
                    case "/": 
                        if (rVal === 0) throw new Error("Runtime Error: Division by zero");
                        return Math.floor(lVal / rVal);
                }
            }
            throw new Error(`Runtime Error: Operator '${op}' requires two arguments.`);
        }

        // Unknown Operator Fallback
        throw new Error(`Runtime Error: Unknown operator structure '${op}'`);
    }

    // internal Error: Unreachable Code
    // If the node is neither number, string, nor array, the AST is corrupt.
    throw new Error(`Internal Error: Unknown AST Node type: ${JSON.stringify(node)}`);
}

## Execution Engine

Since the AST transformer converts the list of statements into a recursive linked list format (`['.', Head, Tail]`), we need an executor to traverse this structure.

The `execute` function:
1.  Checks if the current node is a list node (`.`).
2.  Evaluates and prints the result of the `head` (the current statement).
3.  Recursively calls itself on the `tail` (the remaining statements).
4.  Falls back to evaluating a single node if the structure is not a list.

In [ ]:
function execute(node: AST): void {
    if (node === "" || node === undefined) return;
    
    // Check for Linked List structure ['.', Head, Tail]
    if (Array.isArray(node) && node[0] === '.') {
        const head = node[1];
        const tail = node[2];
        
        // Execute current statement
        if (head !== undefined) {
            try {
                const result = evaluateNode(head);
                console.log(`> ${result}`);
            } catch (e) {
                console.error(`Error: ${(e as Error).message}`);
            }
        }
        
        // Recurse on the rest of the list
        if (tail !== undefined) execute(tail);
        return;
    }
    
    // Fallback: Single Statement execution
    try {
        const result = evaluateNode(node);
        console.log(`> ${result}`);
    } catch (e) {
        console.error(`Error: ${(e as Error).message}`);
    }
}

## Demonstration

We conclude by running a test script that demonstrates variable assignment, operator precedence, and state persistence.

### The Input
The input code performs the following logic:
1.  Assigns $10$ to $x$.
2.  Calculates $y$ using standard precedence ($x * 5$ is evaluated before $+ 2$).
3.  Calculates $z$ using parentheses to override precedence.
4.  Outputs the final value of $z$.

In [ ]:
const inputCode = `
    x := 10;
    y := x * 5 + 2;
    z := (y + 10) / 2;
    z;
`;

### The Transformation Pipeline
To convert the input string into our AST, we import the `cst2ast` function. Its signature is designed to be generic and type-safe:

```typescript
export const cst2ast = (
    cursor: TreeCursor, 
    input: string, 
    operators: Operator[], 
    listVars: string[]
): AST
```

* **`cursor`**: A Lezer `TreeCursor` pointed at the root of the parsed CST.
* **`input`**: The original source code string (required to extract values like numbers and identifiers).
* **`operators`**: A list of tokens (e.g., `+`, `:=`) that should be treated as structural pivots. The transformer promotes these to parent nodes in the AST hierarchy.
* **`listVars`**: A list of grammar variables (e.g., `Program`) that represent sequences. The transformer automatically converts these from flat or recursive CST structures into a standardized recursive linked list (`['.', Head, Tail]`).

In [ ]:
import { cst2ast } from "./CST2AST";

We now parse the code and apply the transformation. Note how `listVars` contains `"Program"`, ensuring our sequence of statements becomes a traversable list.

In [ ]:
const tree = parser.parse(inputCode);
const ast = cst2ast(tree.cursor(), inputCode, operators, listVars);

### Execution
Finally, we run the executor. This will traverse the linked list of statements, evaluate each one, and print the result. We also inspect the environment to verify that variable state was preserved across statements.

In [ ]:
console.log("\n--- Evaluation ---");
execute(ast);
console.log("--- Final Environment ---");
environment;